In [1]:
import telebot
from time import time
import pandas as pd
from os import listdir

In [2]:
token = 'your-api-token' #get here: https://core.telegram.org/bots

In [ ]:
bot = telebot.TeleBot(token, parse_mode='HTML') # You can set parse_mode by default. HTML or MARKDOWN

In [3]:
data_path = '../data/'

In [4]:
# Loadings
df_produto= pd.read_csv(data_path + 'products.csv')
df_order = pd.read_csv(data_path + 'order_products__prior.csv') #apenas prior - fazer teste com a base train
df_depat = pd.read_csv(data_path + 'departments.csv')
df_pedido= pd.read_csv(data_path + 'orders.csv')
df_aisles = pd.read_csv(data_path + 'aisles.csv')

In [5]:
# Joins
df_join=pd.merge(df_order,df_produto,  
                 on='product_id', 
                 how='left')

df_join=pd.merge(df_join,df_depat ,  
                 on='department_id', 
                 how='left')

df_join=pd.merge(df_join,df_pedido ,  
                 on='order_id', 
                 how='left')

In [6]:
def get_most_ordered_products(df_base, user_id):
    return "\n".join(df_base[df_base["user_id"]==user_id].groupby('product_name').count().sort_values('order_id')['order_id'][:15].to_frame().to_string(
        header=False,
        col_space=5
    ).split('\n')[1:])[:-1]+'1'

# test
# print(get_most_ordered_products(df_join,202279))

In [7]:
def get_last_order_products(user_id, df_base):
    last_order_id =  df_base[df_base["user_id"]==user_id]['order_id'].max() ### ordenar por order_number e selecionar order_id
    last_order = df_base[(df_base['user_id']==user_id) & (df_base['order_id'] == last_order_id)]
    return '\n'.join(last_order.groupby('product_name')['product_name'].count().to_frame().to_string(
        header=False,
        col_space=5
    ).split('\n')[1:])[:-1]+'1'


# test
# print(get_last_order_products(202279, df_join))

# Mensagens padrão

In [ ]:
token = '783192310:AAGHRbqknl-gArkYLcbTYmk2necszfIhQuA'
bot = telebot.TeleBot(token, parse_mode='HTML')


@bot.message_handler(commands=['start'])
def churn_message(message):
    chat_id = message.chat.id
    text = f'''
Olá Rafaela! Como vai você?

Verificamos que sua última compra foi há mais de 10 dias, e para te ajudar posso montar seu carrinho.
Se a compra for finalizada nos próximos <b>dois dias</b> a taxa de entrega é por <b>nossa conta</b> :)

Deseja ver as sugestões que eu preparei especialmente pra você?

/status_pedido
/ultimo_pedido
/mais_comprados
/nao
'''
    bot.send_message(chat_id, text)

    
@bot.message_handler(commands=['status_pedido'])
def send_welcome(message):
    chat_id = message.chat.id
    text = """
Sua compra será entregue às <b>15h - 07/09</b>
    """
    bot.send_message(chat_id, text)

    
@bot.message_handler(commands=['ultimo_pedido'])
def churn_yes(message):
    chat_id = message.chat.id
    text = f'''
O que você acha desse carrinho que montei especialmente para você ❤️:

{get_last_order_products(202279, df_join)}

Deseja confirmar o pedido?
/sim
/nao
    '''
    bot.send_message(chat_id, text)
       

@bot.message_handler(commands=['mais_comprados'])
def churn_yes(message):
    chat_id = message.chat.id
    text = f'''
O que você acha desse carrinho que montei especialmente para você ❤️:

{get_most_ordered_products(df_join,202279)}

Deseja confirmar o pedido?
/sim
/nao
    '''
    bot.send_message(chat_id, text)


@bot.message_handler(commands=['sim'])
def send_welcome(message):
    chat_id = message.chat.id
    text = f"""
Estamos quase lá. Confirme a opção de pagamento, por gentileza:
- Cartão de Crédito Mastercard Final 8846

Está correto?
/correto
/mudar_forma_pagamento
    """
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['correto'])
def send_welcome(message):
    chat_id = message.chat.id
    text = f"""
Prontinho :)

Sua entrega será realizada em até dois dias.
    """
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['mudar_forma_pagamento'])
def send_welcome(message):
    chat_id = message.chat.id
    text = f"""
Por questões de segurança, a alteração da forma de pagamento só poder ser feitas pelo site ou aplicativo.

<a>https://www.instacart.com/store/account/payment</a>
    """
    bot.send_message(chat_id, text)
    

@bot.message_handler(commands=['nao'])
def churn_yes(message):
    chat_id = message.chat.id
    text = f'''
Que pena :(

Quando precisar de algo é só chamar me com /start. 
Até breve!
    '''
    bot.send_message(chat_id, text)

bot.polling()